In [28]:
import pandas as pd
import numpy as np
from utils import fuel_algo
import re

In [54]:
def fuel_algo(x):
        # first do everything for MAF non NA
        # then for all MAF NA values, do absLoad calculation
        # Because NA denom or numer => NAN => NA in both => NAN
        sec_hour = 3600
        air_to_fuel = 14.08
        fuel_density = 820
        
        out = pd.DataFrame(np.zeros(shape=(len(x))))
        
        maf_screen = np.array(x['MAF[g/sec]'].isna()).reshape(-1,1)
        maf = x[maf_screen]['MAF[g/sec]']
        fuel_flow = (maf*sec_hour)/(air_to_fuel*fuel_density)
        out[maf_screen] = list(1/(fuel_flow/x[maf_screen]['Vehicle Speed[km/h]']))
        
        VehID = x['VehId'].iloc[0]
        absLoad = x[~maf_screen]['Absolute Load[%]']
        RPM = x[~maf_screen]['Engine RPM[RPM]']
        fuel_flow = x[~maf_screen]['MAF[g/sec]']
        displacement = nonEVs.loc[nonEVs['VehId']==VehID, 'Engine Configuration & Displacement']
        if len(displacement) == 0:
            return pd.DataFrame(np.zeros(shape=(len(x))))
        displacement = displacement.iloc[0]
        displacement = re.findall(r"\d\.\d",displacement)
        if len(displacement) != 1:
            raise Exception('Something Broke in displacement string search')
        displacement = float(displacement[0].strip("L"))
        maf = 1.84 * displacement * absLoad/100 * RPM/2/60
        fuel_flow = (maf*sec_hour)/(air_to_fuel*fuel_density) #update out where MAF is NAN
        out[~maf_screen] = list(1/(fuel_flow/x[~maf_screen]['Vehicle Speed[km/h]']))
        
        out.replace(float('inf'), 0, inplace=True)
        
        return pd.Series(out.iloc[:,0])

In [55]:
trip = pd.read_csv('../sample_HEV_trip.csv')
nonEVs = pd.read_csv('../VED_Static_Data_ICE_HEV.csv')

x = fuel_algo(trip)
print(np.mean(x))


37.24373058326666


In [30]:
trips = pd.read_csv('../alltrips_final.csv')
print(len(trips))
#trips = trips.drop(columns=['Unnamed: 0'])

print(len(trips))
with open('../alltrips_final2.csv', 'w') as f:
    trips.to_csv(f, index_label='TripId')

11803
11803


In [32]:
trips = pd.read_csv('../alltrips_final.csv')
for i, row in trips.iterrows():
    if i == 0:
        print(row['TripId'])
        fname = 'base' + str(int(row['TripId']))
        print(fname)

0.0
base0


In [41]:
aggr = trips['Aggressiveness']
print(np.isinf(aggr).any())
print((aggr == 0).any())
print(aggr.isna().any())


False
False
False
